In [1]:
import numpy as np
import pandas as pd

# To plot
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')

In [2]:
df=pd.read_csv("C:\\Users\\Lenovo\\Desktop\\FyersData\\NIFTY50-INDEX_15min_2017-05-12_to_2022-06-30.csv")


In [3]:
df

,Unnamed: 0,date,time,open,high,low,close,volume
0,0,2017-07-17,09:00,9908.15,9908.15,9908.15,9908.15,0
1,1,2017-07-17,09:15,9908.15,9913.90,9899.50,9911.65,0
2,2,2017-07-17,09:30,9911.95,9915.15,9894.85,9897.70,0
3,3,2017-07-17,09:45,9899.05,9916.25,9898.45,9915.35,0
4,4,2017-07-17,10:00,9915.10,9916.70,9907.15,9915.30,0
...,...,...,...,...,...,...,...,...
30702,30702,2022-06-30,13:45,15830.85,15840.15,15807.60,15810.95,0
30703,30703,2022-06-30,14:00,15812.55,15846.65,15796.40,15846.00,0
30704,30704,2022-06-30,14:15,15845.15,15852.40,15831.35,15838.40,0
30705,30705,2022-06-30,14:30,15837.90,15838.45,15793.60,15802.25,0


In [4]:
DMIinterval=13
adxLevel=23.5

In [5]:
def getMA(series, maperiod ,ema):
    
    
    if ema == True:
        # Use exponential moving average
        alpha=1/(maperiod) ###RMA
        #alpha=2/(maperiod+1) ###EMA
        ma = series.ewm(alpha=alpha,adjust=True, min_periods = maperiod).mean()
        #ma = series.ewm(com = maperiod - 1, adjust=True, min_periods = maperiod).mean()
        
    else:
        # Use simple moving average
        ma = series.rolling(window = maperiod).mean()
        
    return ma


def MA(series, maperiod ):
    
    alpha=1/(maperiod) ###RMA
    #alpha=2/(maperiod+1) ###EMA
    ma = series.ewm(alpha=alpha).mean()
    return ma


In [6]:

def StochRSI_EMA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = 100*((rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min()))
    stochrsi_K = (stochrsi.ewm(span=smoothK).mean())
    stochrsi_D = (stochrsi_K.ewm(span=smoothD).mean())

    return stochrsi, stochrsi_K, stochrsi_D

In [7]:
def adx(df,maperiod):
    
    for i in range(1,len(df)):
        df.loc[i,'high_low']=df.loc[i,'high']-df.loc[i,'low']
        df.loc[i,'high_preClose']=abs(df.loc[i,'high']-df.loc[i-1,'close'])
        df.loc[i,'low_preClose']=abs(df.loc[i,'low']-df.loc[i-1,'close'])
        df.loc[i,'trueRange']=max(df.loc[i,'high_low'],df.loc[i,'high_preClose'],df.loc[i,'low_preClose'])
        
        
        if(((df.loc[i,'high']-df.loc[i-1,'high'])>(df.loc[i-1,'low']-df.loc[i,'low']))  and (df.loc[i,'high']-df.loc[i-1,'high'] >0)):
            df.loc[i,'plusdm']=(df.loc[i,'high']-df.loc[i-1,'high'])
        else:
            df.loc[i,'plusdm']=0

        if(((df.loc[i-1,'low']-df.loc[i,'low'])>(df.loc[i,'high']-df.loc[i-1,'high'])) and(df.loc[i-1,'low']-df.loc[i,'low'] >0)):
            df.loc[i,'minusdm']=(df.loc[i-1,'low']-df.loc[i,'low'])
        else:
            df.loc[i,'minusdm']=0
    
    # print(df)
    
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['smoothed+dm']=getMA(df['plusdm'],maperiod,1)
    df['smoothed-dm']=getMA(df['minusdm'],maperiod,1)
    df['smoothedTR']=getMA(df['trueRange'],maperiod,1)
    # df.to_csv('data.csv')
    for i in range(0,len(df)):
        df.loc[i,'plus_di']= (df.loc[i,'smoothed+dm']/df.loc[i,'smoothedTR'])*100
        df.loc[i,'minus_di']= (df.loc[i,'smoothed-dm']/df.loc[i,'smoothedTR'])*100
        df.loc[i,'DX']= (abs(df.loc[i,'smoothed+dm']-df.loc[i,'smoothed-dm'])/(df.loc[i,'smoothed+dm']+df.loc[i,'smoothed-dm']))*100

    df['adx']=getMA(df['DX'],maperiod,1)
    return df


In [8]:
def getHalfTrend(amplitude=2):
    global df
    
    trend=1
    nextTrend=0
    
    df['highma']=getMA(df['high'],amplitude,0)
    df['lowma']=getMA(df['low'],amplitude,0)
    df['highPrice']=df['high'].rolling(amplitude).max()
    df['lowPrice']=df['low'].rolling(amplitude).min() 
    
    maxlowPrice=df.loc[0,'low']
    minHighPrice=df.loc[0,'high']

    for i in range(amplitude-1,len(df)):
        
        

        # maxhigh=df['high'].rolling(amplitude).max()
        # minlow=df['high'].rolling(amplitude).min()
        
            
        # highPrice=maxhigh
        # lowPrice=minlow

        highPrice=df.loc[i,'highPrice']
        lowPrice=df.loc[i,'lowPrice']
        
        highma=df.loc[i,'highma']
        lowma=df.loc[i,'lowma']

        

        if nextTrend==1:
            maxlowPrice=max(lowPrice,maxlowPrice)

            if((highma<maxlowPrice) and (df.loc[i,'close']<df.loc[i-1,'low'])):
                trend=-1
                nextTrend=0
                minHighPrice=highPrice
        else:
            minHighPrice=min(highPrice,minHighPrice) 

            if((lowma>minHighPrice) and (df.loc[i,'close']>df.loc[i-1,'high'])):
                trend=1
                nextTrend=1
                maxlowPrice=lowPrice

        df.loc[i,'trend']=trend
        

In [13]:
# df['stoch_rsi'],df['stochrsi_K'],df['stochrsi_D']=StochRSI_EMA(df['close'])

df=adx(df,DMIinterval)
getHalfTrend()
df.dropna(inplace=True)
df.reset_index(inplace=True)
df=df[['date','time','high','low','close','stochrsi_K','stochrsi_D','plus_di','minus_di','adx']]


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7372\1367956545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i,'high_low']=df.loc[i,'high']-df.loc[i,'low']
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7372\1367956545.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i,'high_preClose']=abs(df.loc[i,'high']-df.loc[i-1,'close'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7372\1367956545.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [14]:
for i in df.index:
    ### Half Trend 
    if((df.loc[i,'adx']>adxLevel and (df.loc[i,'plus_di']>df.loc[i,'minus_di']) and (df.loc[i,'trend']==1))):
    ### Without Half Trend
#     if((df.loc[i,'adx']>adxLevel and (df.loc[i,'plus_di']>df.loc[i,'minus_di']) )):
        df.loc[i,'signal']=1


    elif((df.loc[i,'adx']>adxLevel and (df.loc[i,'minus_di']>df.loc[i,'plus_di']) and (df.loc[i,'trend']==-1))):
#     elif((df.loc[i,'adx']>adxLevel and (df.loc[i,'minus_di']>df.loc[i,'plus_di']) )):
        df.loc[i,'signal']=-1
    else:
        df.loc[i,'signal']=0
    

print(df) 
df.to_csv('data.csv')

Empty DataFrame
Columns: [date, time, high, low, close, stochrsi_K, stochrsi_D, plus_di, minus_di, adx]
Index: []


In [ ]:
df=df.round(2)

In [ ]:
df

### Stoch RSI + HT + DMI (with + without HalfTrend)

In [ ]:
import datetime
from time import time

In [ ]:
buy=None
sell=None
#stores return for each trade

Return=0
max_sl_point=None
max_sl_pecentage=.18/100
max_Drawdown=0
no_of_trades=0
no_of_Profitable_trades=0
no_of_loosing_trades=0
max_opposite=0.0

after_max=0.0
after_min=0.0
entry=0.0


column_names = ["date","entry_time","exit_time","entry","exit", "tradetype","return","after_max","after_min","max_sl","max_sl"]
trades = pd.DataFrame(columns = column_names)
cnt=-1




for i in range(1,len(df)):
    
    
    if(df.loc[i,'time'] >str(datetime.time(15,10))):
       
        if(buy and (buy-df.loc[i,'low']>max_sl_point)):
            trades.loc[cnt,'exit']=buy-max_sl_point
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'return']=-max_sl_point
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            no_of_loosing_trades+=1
            no_of_trades+=1
            buy=None
        elif(sell and (df.loc[i,'high']-sell>max_sl_point)):
            trades.loc[cnt,'exit']=sell+max_sl_point
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'return']=-max_sl_point
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            no_of_loosing_trades+=1
            no_of_trades+=1
            sell=None
            
        if(buy):
            
            if df.loc[i,'close'] >buy:
                no_of_Profitable_trades+=1

            else:
                no_of_loosing_trades+=1
                max_Drawdown=max(max_Drawdown,(buy-df.loc[i,'close']))
            
            if(df.loc[i,'low']<buy):
                    max_opposite=max(max_opposite,buy-df.loc[i,'low'])
            no_of_trades+=1
            capital=capital+(df.loc[i,'close']-buy)
            Return=Return+(df.loc[i,'close']-buy)
            
            print("Buy exit:",df.loc[i,'date'],df.loc[i,'time'])
            print("Return: ",round(df.loc[i,'close']-buy,2))
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'exit']=df.loc[i,'close']
            trades.loc[cnt,'return']=round(df.loc[i,'close']-buy,2)
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            buy=None
            
            
        elif(sell):
            
            if (df.loc[i,'close']) < sell:
                no_of_Profitable_trades+=1

            else:
                no_of_loosing_trades+=1
                max_Drawdown=max(max_Drawdown,(df.loc[i,'close']-sell))

            if(df.loc[i,'high']>sell):
                    max_opposite=min(max_opposite,sell-df.loc[i,'high'])
            no_of_trades+=1
            capital=capital+(sell-df.loc[i,'close'])
            Return=Return+(sell-df.loc[i,'close'])
            
            print("Sell exit:",df.loc[i,'date'],df.loc[i,'time'])
            print("Return: ",round(sell-df.loc[i,'close'],2))
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'exit']=df.loc[i,'close']
            trades.loc[cnt,'return']=round(sell-df.loc[i,'close'],2)
            
            sell=None
    
    
    
    
    elif(df.loc[i,'time'] >str(datetime.time(9,40))):


        if(buy):
            after_max=max(after_max,df.loc[i,'high']-buy)
            after_min=min(after_min,df.loc[i,'low']-buy)

        elif(sell):
            after_max=max(after_max,df.loc[i,'high']-sell)
            after_min=min(after_min,df.loc[i,'low']-sell)
        

        if(buy and (buy-df.loc[i,'low']>max_sl_point)):
            trades.loc[cnt,'exit']=buy-max_sl_point
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'return']=-max_sl_point
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            no_of_loosing_trades+=1
            no_of_trades+=1
            buy=None
        elif(sell and (df.loc[i,'high']-sell>max_sl_point)):
            trades.loc[cnt,'exit']=sell+max_sl_point
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'return']=-max_sl_point
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            no_of_loosing_trades+=1
            no_of_trades+=1
            sell=None


        elif(df.loc[i,'signal']!=1) and buy:

            if df.loc[i,'close'] >buy:
                no_of_Profitable_trades+=1

            else:
                no_of_loosing_trades+=1
                max_Drawdown=max(max_Drawdown,(buy-df.loc[i,'close']))
            
            if(df.loc[i,'low']<buy):
                    max_opposite=max(max_opposite,buy-df.loc[i,'low'])
            no_of_trades+=1
            capital=capital+(df.loc[i,'close']-buy)
            Return=Return+(df.loc[i,'close']-buy)
            
            print("Buy exit:",df.loc[i,'date'],df.loc[i,'time'])
            print("Return: ",round(df.loc[i,'close']-buy,2))
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'exit']=df.loc[i,'close']
            trades.loc[cnt,'return']=round(df.loc[i,'close']-buy,2)
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min

            buy=None

        elif(df.loc[i,'signal']!=-1) and sell:

            if (df.loc[i,'close']) < sell:
                no_of_Profitable_trades+=1

            else:
                no_of_loosing_trades+=1
                max_Drawdown=max(max_Drawdown,(df.loc[i,'close']-sell))

            if(df.loc[i,'high']>sell):
                    max_opposite=min(max_opposite,sell-df.loc[i,'high'])
            no_of_trades+=1
            capital=capital+(sell-df.loc[i,'close'])
            Return=Return+(sell-df.loc[i,'close'])
            
            print("Sell exit:",df.loc[i,'date'],df.loc[i,'time'])
            print("Return: ",round(sell-df.loc[i,'close'],2))
            
            trades.loc[cnt,'exit_time']=df.loc[i,'time']
            trades.loc[cnt,'exit']=df.loc[i,'close']
            trades.loc[cnt,'return']=round(sell-df.loc[i,'close'],2)
            trades.loc[cnt,'after_max']=after_max
            trades.loc[cnt,'after_min']=after_min
            
            sell=None
        
        if ((df.loc[i,'signal']==1) and not buy):
            
            buy=df.loc[i,'close']
            entry=buy
            max_sl_point=max_sl_pecentage*buy
            print("Buy:",df.loc[i,'date'],df.loc[i,'time'])
            dict1 = {"date":df.loc[i,'date'],"entry_time":df.loc[i,'time'],'entry':entry,'tradetype': 'long','max_sl':max_sl_point}
            trades = trades.append(dict1, ignore_index = True)
            cnt+=1
            after_max=0.0
            after_min=0.0
            
        elif((df.loc[i,'signal']==-1) and not sell):
            sell=df.loc[i,'close']
            entry=sell
            max_sl_point=max_sl_pecentage*sell
            print("Sell:",df.loc[i,'date'],df.loc[i,'time'])
            dict1 = {"date":df.loc[i,'date'],"entry_time":df.loc[i,'time'],'entry':entry,'tradetype': 'sell','max_sl':max_sl_point}
            trades = trades.append(dict1, ignore_index = True)
            cnt+=1
            after_max=0.0
            after_min=0.0
            
            
    

In [ ]:
print("no_of_trades:",no_of_trades)
print("no_of_Profitable_trades:",no_of_Profitable_trades)
print("no_of_loosing_trades:",no_of_loosing_trades)
print("Max Drawdown:",round(max_Drawdown,2))
print("No of points cap:",round(trades['return'].sum(),2))

In [ ]:
trades=trades.round(2)
trades.to_csv('Trades2017.csv')

In [ ]:
12933.55/3605

In [ ]:
.18*17000
